In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-190310
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-190310


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "canina-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

A new compute target has been created.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.05, 10),
    "max_iter": choice(100, 300, 500, 700, 1000)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=3)

if "training" not in os.listdir():
    os.mkdir("./training")

#### Copy train.py script into "./training" to use it in ScriptRunConfig object.
import shutil

shutil.copy('train.py', "./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    max_concurrent_runs=4)

# SKIP

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.05, 10),
    "max_iter": choice(100, 300, 500, 700, 1000)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=3)

if "training" not in os.listdir():
    os.mkdir("./training")

#### Copy train.py script into "./training" to use it in ScriptRunConfig object.
import shutil

shutil.copy('train.py', "./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=24,
                                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()
# Print best metrics 
best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

# save the model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrivemodel', model_path = 'outputs/model.joblib')

RunId: HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7
Web View: https://ml.azure.com/runs/HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-189698/workspaces/quick-starts-ws-189698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7
Web View: https://ml.azure.com/runs/HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-189698/workspaces/quick-starts-ws-189698&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Run Id :  HD_1226ca73-fbfe-4afa-bbee-12d1798f48e7_23
Metrics : {'Regularization Strength:': 6.463739391157785, 'Max iterations:': 300, 'Accuracy': 0.9095599393019727}


In [7]:
# This print is to see the best model
print(best_hyperdrive_model.name, best_hyperdrive_model.id, best_hyperdrive_model.version, sep='\t')

hyperdrivemodel	hyperdrivemodel:1	1


In [9]:
best_run_hyperdrive.get_file_names()

['logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [13]:
best_run_hyperdrive.download_file(best_run_hyperdrive.get_file_names()[3],
                                 output_file_path = './outputs/')

In [14]:
joblib.load('./outputs/model.joblib')

LogisticRegression(C=6.463739391157785, max_iter=300)

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path_data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path_data, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')
    

In [7]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split clean data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

ds_train = x_train.join(y_train)

In [8]:
# Register the dataset
import pandas as pd
from azureml.core.dataset import Dataset
from azureml.core import Datastore

ds_train.to_csv("./data_train.csv", index=False)
df_train = pd.read_csv('data_train.csv')

datastore = Datastore.get(ws, 'workspaceblobstore')
data_train = Dataset.Tabular.register_pandas_dataframe(df_train, datastore, "traindataset", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/01bb62c5-493b-4268-95b7-cc2fb71af314/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "primary_metric" : 'accuracy',
    "featurization" : 'auto',
    "verbosity" : logging.INFO,
    "n_cross_validations" : 5,
    "enable_onnx_compatible_models" : True
}

# Split data into train and test sets using parameter test_size.

automl_config = AutoMLConfig(
    task='classification',
    debug_log='automl_errors.log',
    compute_target=compute_target,
    training_data=data_train,
    label_column_name='y',
    **automl_settings)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
run_automl = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on canina-cluster with default configuration
Running on remote compute: canina-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5c81dce4-77ac-4f34-9dfc-35c61973b7dc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [14]:
run_automl.wait_for_completion(show_output=False)

{'runId': 'AutoML_5c81dce4-77ac-4f34-9dfc-35c61973b7dc',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-28T18:36:13.506965Z',
 'endTimeUtc': '2022-03-28T19:04:03.29991Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'canina-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-190310","workspace_name":"quick-starts-ws-190310","region":"southcentralus","compute_target":"canina-cl

In [15]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###



best_run_automl, best_automl_model = run_automl.get_output()

automl_model_name = best_run_automl.properties["model_name"]
registered_model = automl_run.register_model(model_name=automl_model_name,
                                                 description = "Automl model",
                                                 tags=None)


In [41]:
registered_model

Model(workspace=Workspace.create(name='quick-starts-ws-190310', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-190310'), name=AutoML5c81dce4730, id=AutoML5c81dce4730:1, version=1, tags={}, properties={})

In [16]:
# List all files to search for "model.pkl"
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'logs/azureml/azureml_automl.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/engineered_feature_names.json',
 'outputs/env_dependencies.json',
 'outputs/featurization_summary.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.onnx',
 'outputs/model.pkl',
 'outputs/model_onnx.json',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py',
 'outputs/scoring_file_v_2_0_0.py',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [39]:
best_run_automl.get_file_names()[43]

'outputs/pipeline_graph.json'

In [29]:
best_run_automl.get_file_names()[41]

'outputs/model.pkl'

In [30]:
best_run_automl.download_file(best_run_automl.get_file_names()[41],
                                 output_file_path = './outputs/')

In [31]:
best_run_automl.download_file(best_run_automl.get_file_names()[40],
                                 output_file_path = './outputs/')

In [40]:
best_run_automl.download_file(best_run_automl.get_file_names()[43],
                                 output_file_path = './outputs/')

In [33]:
# Test the performance of the automl model.
run_metrics = best_run_automl.get_metrics()
for m, val in run_metrics.items():
    print(f"{m}: {val}")

f1_score_micro: 0.9180955993930198
balanced_accuracy: 0.7734887827751644
recall_score_weighted: 0.9180955993930198
precision_score_micro: 0.9180955993930198
norm_macro_recall: 0.5469775655503286
AUC_micro: 0.980787518622735
precision_score_macro: 0.7983905483319128
average_precision_score_micro: 0.9815173419581903
AUC_macro: 0.9474793301455173
log_loss: 0.23836500787090098
weighted_accuracy: 0.9539720201055791
matthews_correlation: 0.5712442505886312
average_precision_score_macro: 0.8269887668990032
accuracy: 0.9180955993930198
recall_score_micro: 0.9180955993930198
AUC_weighted: 0.9474793301455172
recall_score_macro: 0.7734887827751644
f1_score_weighted: 0.916301686461009
precision_score_weighted: 0.9149692298057974
average_precision_score_weighted: 0.9559045326238907
f1_score_macro: 0.7850406136198448
accuracy_table: aml://artifactId/ExperimentRun/dcid.AutoML_5c81dce4-77ac-4f34-9dfc-35c61973b7dc_30/accuracy_table
confusion_matrix: aml://artifactId/ExperimentRun/dcid.AutoML_5c81dce4-7

In [37]:
best_run_automl.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":3}',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_5c81dce4-77ac-4f34-9dfc-35c61973b7dc_ModelExplain',
 'model_explanation': 'True'}

### Delete the compute cluster "canina-cluster"

In [42]:
# Remove the compute cluster following training.
compute_target.delete()